<a href="https://colab.research.google.com/github/nicpier/dowjones-replica/blob/main/fantafoot_moneyball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fantasy football strategy, Moneyball style**

Fantasy football is a social game based on football (soccer, for Americans), which consists of building and managing virtual teams made up of real football players, chosen from among those participating in the tournament to which the game refers. [Check here for more information](https://en.wikipedia.org/wiki/Fantasy_football_(association)).

Although this strategy is based on the Italian version of the game (the reference tournament is Serie A, the top football league in Italy), I prefer to write it down in English to make it more accessible to an international audience (as well as to keep away my Italian friends and opponents).

If you don't feel comfortable with the rules of the game, here you can find pretty much [all the essentials about Italian fantasy football](https://www.fantacalcio.it/regolamenti) by one of the greatest game hosts (sorry, it is in Italian only).

# **A Moneyball approach**

The core approach of this strategy comes from the events described in the book [Moneyball: The Art of Winning an Unfair Game](https://en.wikipedia.org/wiki/Moneyball), and later in the film of the same name. The principle underlying the method goes like this: "buy victories, not players". It means that to have the highest possible probability to win you should shift your focus and start considering players as if they were just buckets of bonuses, and therefore scorable points.

If such an abstraction of reality seems extreme in football to you, you forgot that there have already been some clubs that have adopted variants of this approach. [The latest arrival is precisely AC Milan](https://www.gazzetta.it/Calcio/Serie-A/Milan/08-06-2023/milan-moneyball-club-billy-beane-4601940464413_preview.shtml?reason=unauthenticated&origin=http%3A%2F%2Fwww.gazzetta.it%2FCalcio%2FSerie-A%2FMilan%2F08-06-2023%2Fmilan-moneyball-club-billy-beane-4601940464413.shtml), a very popular team in Italy. So, if you decide to implement this strategy, this season you will have the opportunity to see from both the real-world and fantasy-world sides what it can yield.

# **Steps to follow**

1-calculate total number of football players that will be bought by position (G=goalkeeper, D=defender, M=midfielder, A=attacker)

2-estimate total goals scored by those top players in the league this year using stats or ML or other (why goals? because it is the most important kind of bonus)

3-calculate season total goal by summing the expected goals belonging to the top players and determine what is the lowest threshold to have mathematical victory

4-calculate average expected goals per team in game, per tournament day

5-calculate average expected points per team in game, per tournament day

6-calculate threshold points to win the day mathematically, and convert to goal needs

7-calculate the lowest goals threshold to win the tournament mathematically

8-knowing the starting budget, determine how much a goal is in term of fantasy credits

9-at the auction, buy goals buckets without exceding the value previously determined. The purpose is to reach the goal threshold to mathematically win the tournament, or at least to get as closer as possible to it

10-manage your team with player exchanges during the season in order to further increase your amount of goals

...work in progress to be finished

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = 'https://raw.githubusercontent.com/nicolapieri/fantafoot-moneyball/main/data/'

listone = pd.read_csv(path + 'listone.csv', sep=';')[['Id', 'Nome', 'R', 'Squadra']]

stats21 = pd.read_csv(path + 'stats20-21.csv', sep=';')[['Id', 'Pv', 'Mv', 'Mf', 'Gf', 'Ass','Gs', 'Rp', 'Rc', 'R+', 'R-', 'Amm', 'Esp', 'Au']]
stats21.rename(columns={'Pv': 'Pv21',
                        'Mv': 'Mv21',
                        'Mf': 'Mf21',
                        'Gf': 'Gf21',
                        'Ass': 'Ass21',
                        'Gs': 'Gs21',
                        'Rp': 'Rp21',
                        'Rc': 'Rc21',
                        'R+': 'R+21',
                        'R-': 'R-21',
                        'Amm': 'Amm21',
                        'Esp': 'Esp21',
                        'Au': 'Au21'},
               inplace=True)

stats22 = pd.read_csv(path + 'stats21-22.csv', sep=';')[['Id', 'Pv', 'Mv', 'Mf', 'Gf', 'Ass','Gs', 'Rp', 'Rc', 'R+', 'R-', 'Amm', 'Esp', 'Au']]
stats22.rename(columns={'Pv': 'Pv22',
                        'Mv': 'Mv22',
                        'Mf': 'Mf22',
                        'Gf': 'Gf22',
                        'Ass': 'Ass22',
                        'Gs': 'Gs22',
                        'Rp': 'Rp22',
                        'Rc': 'Rc22',
                        'R+': 'R+22',
                        'R-': 'R-22',
                        'Amm': 'Amm22',
                        'Esp': 'Esp22',
                        'Au': 'Au22'},
               inplace=True)

stats23 = pd.read_csv(path + 'stats22-23.csv', sep=';')[['Id', 'Pv', 'Mv', 'Fm', 'Gf', 'Ass','Gs', 'Rp', 'Rc', 'R+', 'R-', 'Amm', 'Esp', 'Au']]
stats23.rename(columns={'Pv': 'Pv23',
                        'Mv': 'Mv23',
                        'Fm': 'Mf23',
                        'Gf': 'Gf23',
                        'Ass': 'Ass23',
                        'Gs': 'Gs23',
                        'Rp': 'Rp23',
                        'Rc': 'Rc23',
                        'R+': 'R+23',
                        'R-': 'R-23',
                        'Amm': 'Amm23',
                        'Esp': 'Esp23',
                        'Au': 'Au23'},
               inplace=True)

In [ ]:
listone

,Id,Nome,R,Squadra
0,572,Meret,P,Napoli
1,4312,Maignan,P,Milan
2,2814,Provedel,P,Lazio
3,453,Szczesny,P,Juventus
4,4270,Rui Patricio,P,Roma
...,...,...,...,...
511,6242,Bidaoui,A,Frosinone
512,6243,Borrelli,A,Frosinone
513,6247,Yalcin,A,Genoa
514,6249,Diaw,A,Monza


In [ ]:
stats21

,Id,Pv21,Mv21,Mf21,Gf21,Ass21,Gs21,Rp21,Rc21,R+21,R-21,Amm21,Esp21,Au21
0,350,37,6.28,5.32,0,0,38,1,0,0,0,1,0,0
1,453,30,6.22,5.38,0,1,32,2,0,0,0,0,0,0
2,250,37,6.03,5.23,0,0,33,2,0,0,0,1,0,1
3,610,25,6.10,5.26,0,0,26,2,0,0,0,0,1,0
4,761,37,6.38,5.11,0,1,54,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,5361,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0
674,5378,3,6.17,6.83,0,2,0,0,0,0,0,0,0,0
675,5414,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0
676,5419,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0


In [ ]:
stats22

,Id,Pv22,Mv22,Mf22,Gf22,Ass22,Gs22,Rp22,Rc22,R+22,R-22,Amm22,Esp22,Au22
0,4312,32,6.27,5.62,0,1,21,0,0,0,0,1,0,0
1,453,33,6.12,5.48,0,0,29,3,0,0,0,2,0,0
2,2468,31,6.26,5.44,0,0,25,0,0,0,0,1,0,0
3,250,37,6.16,5.34,0,0,30,0,0,0,0,1,0,0
4,316,10,6.10,5.55,0,0,8,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,5472,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0
671,5493,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0
672,5505,2,5.75,5.75,0,0,0,0,0,0,0,0,0,0
673,5512,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0


In [ ]:
stats23

,Id,Pv23,Mv23,Mf23,Gf23,Ass23,Gs23,Rp23,Rc23,R+23,R-23,Amm23,Esp23,Au23
0,572,34,6.18,5.56,0,0,24,1,0,0,0,0,0,0
1,2814,38,6.26,5.46,0,0,30,0,0,0,0,1,0,0
2,4312,22,6.16,5.34,0,0,21,1,0,0,0,0,0,0
3,4964,31,6.40,5.19,0,0,39,1,0,0,0,3,0,0
4,453,28,6.11,5.18,0,0,26,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,6063,1,6.00,6.00,0,0,0,0,0,0,0,0,0,0
668,6113,5,5.70,5.60,0,0,0,0,0,0,0,1,0,0
669,6143,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0
670,6160,1,6.00,6.00,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 1-selezione top giocatori acquistabili per max(gol) o min(gol subiti) --> 10*3P + 10*8D + 10*8C + 10*6A
# 2-previsione con AI gol segnati in questa stagione --> Features: Pv, Ass, Rc, ecc.
# 3-selezione top per ruolo sulla base dei titolari previsti (modulo 4-3-3) --> 10P + 40D + 30C + 30A
# 4-stabilire fair value dei soli titolari in crediti sulla base dei gol e pesati per ruolo --> 4 c/gol/D, 6 c/gol/C, 8 c/gol/A
# 5-acquistare gol attraverso i giocatori, puntando ad ottimizzare il monte crediti --> target indicativo 130 gol totali